# Demonstrations of Sifter Precalc/SQL Functionality

 - This notebook contains a mixture of demos of finished code, as well as WIP experimentation with ways to speed-up the code 

In [10]:
import time
import numpy as np
import scipy.stats as stats
import math
import random
from collections import defaultdict
import os
import sys
from collections import Counter
import glob 
import warnings
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import importlib
from astropy.time import Time

# -------------------------------------------------------------------------------------
# Local imports
# -------------------------------------------------------------------------------------
sys.path.append( os.path.join(os.path.dirname(os.getcwd() ), 'sifter')  )
sys.path.append( os.path.join(os.path.dirname(os.getcwd() ), 'tests')  )
sys.path.append( os.path.join( os.path.dirname(os.path.dirname(os.getcwd()) ), 'orbit_cheby', 'orbit_cheby')  )
import orbit_cheby
import precalc 
import sql 
import test_tracklets 

importlib.reload(orbit_cheby)
importlib.reload(precalc)
importlib.reload(test_tracklets)
importlib.reload(sql)


<module 'sql' from '/Users/matthewjohnpayne/Envs/sifter/sifter/sql.py'>

# Explicit Test Code Exists in sifter/tests

 - test_sqlite.py
 - test_tracklets.py

# Top Level: "Tracklets" - Storage of obs80-strings for multiple tracklets
 - This takes input sets of obs80-strings (assumed to have already been partitioned into subsets, one subset per tracklet), performs some useful precalculations, and then saves them into an sql database
 - This calls a number of under-lying functionalities

In [20]:
####--- Here I set up an empty database and create some fake observations ---###

# Create db from scratch (uses convenience funciton from test-suite)
test_tracklets.convenience_func_create_db_and_tables()

# define observations
# *** AT PRESENT THESE ARE JUST DUMMY/BLANK OBS ***
observation_pairs= [[ '     K11Q88F*~C2011 08 29.52378 01 57 34.729+14 35 44.64         22.8 rc~0qBd568',
                     '     K11Q88F ~C2011 08 29.55470 01 57 34.343+14 35 42.61         22.9 rc~0qBd568',
                     '     K11Q88F ~C2011 08 29.61470 01 57 34.343+14 35 42.59         22.9 rc~0qBd568'],
                    [ '     K11Q99F*~C2012 08 29.52378 01 57 34.729+14 35 44.64         22.8 rc~0qBd568',
                     '     K11Q99F ~C2012 08 29.58470 01 57 34.343+14 35 42.61         22.9 rc~0qBd568',
                     '     K11Q99F ~C2012 08 29.61470 01 57 34.343+14 35 42.59         22.9 rc~0qBd568']]



In [21]:
%%time 
# instantiate with observation_pair
T = precalc.Tracklets( observation_pairs )



Received more than 2 observations.
Only using first and last one.
We should also pre-calc and save the observatory location for each observation
Received more than 2 observations.
Only using first and last one.
We should also pre-calc and save the observatory location for each observation
CPU times: user 14.1 ms, sys: 3.42 ms, total: 17.5 ms
Wall time: 18.5 ms


In [22]:
# test that the above caused the tracklet to be uploaded to db
cur = T.conn.cursor()
cur.execute('SELECT * from tracklets')
f = cur.fetchone()
assert( len(f)>3 ), 'data not uploaded'
if len(f) > 3 : print('Data uploaded as required')

# Completely delete db to facilitate future testing
os.remove(sql.fetch_db_filepath())


Data uploaded as required


# Lower-Level Functionalities
 - The above "Tracklets" instantiation uses some underlying functionality to perform the parsing & storage of the obs80 strings
 - Here we demonstrate some of the underlying functionality

##### sql.create_connection
 - Create a connection to an extant database 

In [27]:
# Where do we want the db to live
assert 'sifter' in sql.fetch_db_filepath()

if os.path.isfile( sql.fetch_db_filepath() ):
    os.remove(sql.fetch_db_filepath())

# Does a db get created & connected-to ?
conn = sql.create_connection( sql.fetch_db_filepath() )
assert os.path.isfile( os.path.join( sql.fetch_db_filepath() ) )


##### precalc.parse_observation_list
 - parse the list of observations for a single tracklet, perform calculations of (e.g.) HP, RoM, etc, and then creates an summary dictionary 

In [23]:
# define observations
# *** AT PRESENT THESE ARE JUST DUMMY/BLANK OBS ***
observation_pairs= [[ '     K11Q88F*~C2011 08 29.52378 01 57 34.729+14 35 44.64         22.8 rc~0qBd568',
                     '     K11Q88F ~C2011 08 29.55470 01 57 34.343+14 35 42.61         22.9 rc~0qBd568',
                     '     K11Q88F ~C2011 08 29.61470 01 57 34.343+14 35 42.59         22.9 rc~0qBd568'],
                    [ '     K11Q99F*~C2012 08 29.52378 01 57 34.729+14 35 44.64         22.8 rc~0qBd568',
                     '     K11Q99F ~C2012 08 29.58470 01 57 34.343+14 35 42.61         22.9 rc~0qBd568',
                     '     K11Q99F ~C2012 08 29.61470 01 57 34.343+14 35 42.59         22.9 rc~0qBd568']]
    
# call parse_observations
tracklet_dictionary_list = T.parse_observation_lists( observation_pairs )


# check that the returned results are as expected
assert isinstance(tracklet_dictionary_list, list)
assert len(observation_pairs) == len(tracklet_dictionary_list)
for tracklet_dictionary in tracklet_dictionary_list:
    assert 'JD' in tracklet_dictionary
    assert 'HP' in tracklet_dictionary
    assert 'tracklet_name' in tracklet_dictionary


Received more than 2 observations.
Only using first and last one.
We should also pre-calc and save the observatory location for each observation
Received more than 2 observations.
Only using first and last one.
We should also pre-calc and save the observatory location for each observation


##### precalc.save_tracklets & sql.upsert_tracklets
 - Saves a dictionary into a SQLITE db, using JD & HP as important columns for selection/query later-on

In [25]:
# Create db from scratch
test_tracklets.convenience_func_create_db_and_tables()


# Set up a Tracklet and use the parse_observations routine to get JD, HP, ...
T = precalc.Tracklets()
observation_pairs= [[ '     K11Q88F*~C2011 08 29.52378 01 57 34.729+14 35 44.64         22.8 rc~0qBd568',
                     '     K11Q88F ~C2011 08 29.55470 01 57 34.343+14 35 42.61         22.9 rc~0qBd568',
                     '     K11Q88F ~C2011 08 29.61470 01 57 34.343+14 35 42.59         22.9 rc~0qBd568'],
                    [ '     K11Q99F*~C2012 08 29.52378 01 57 34.729+14 35 44.64         22.8 rc~0qBd568',
                     '     K11Q99F ~C2012 08 29.58470 01 57 34.343+14 35 42.61         22.9 rc~0qBd568',
                     '     K11Q99F ~C2012 08 29.61470 01 57 34.343+14 35 42.59         22.9 rc~0qBd568']]
    
# call parse_observations
tracklet_dictionary_list = T.parse_observation_lists( observation_pairs )

# Now save the data in the db
T.save_tracklets(tracklet_dictionary_list)

# Test the data was uploaded and can be downloaded
cur = T.conn.cursor()
cur.execute('SELECT * from tracklets')
f = cur.fetchall()
assert( len(f)==2 and np.all([ len(_)>3 for _ in f]) ), 'data not uploaded'

# Completely delete db to facilitate future testing
os.remove(sql.fetch_db_filepath())



Received more than 2 observations.
Only using first and last one.
We should also pre-calc and save the observatory location for each observation
Received more than 2 observations.
Only using first and last one.
We should also pre-calc and save the observatory location for each observation
